# 실습: scikit-learn을 활용한 텍스트 분석 실습

## 실습 목표

인간에 의해 생성된 데이터의 대부분은 자유롭게 작성된(구조화되지 않은) 텍스트로 구성되어 있습니다. 이러한 텍스트를 적절한 형태로 가공하고 그것으로부터 유용한 정보를 추출 해낼 수 있다면 데이터의 활용성은 극대화될 것입니다. 본 실습에서는 `scikit-learn`, `pandas` 등의 파이썬 라이브러리를 활용하여 SMS 메시지 데이터를 적절한 형태로 가공하고 자질(feature)를 추출한 후, 최종적으로 특정 매시지가 스팸인지 아닌지 여부를 판별하는 기계학습(machine learning) 모델을 생성하는 것을 목표로 합니다.

## 실습 구성

1. `scikit-learn` 맛보기
2. 텍스트를 연산 가능한 형태로 변환하기
3. `pandas`를 활용하여 텍스트 데이터셋 구조화하기
4. 데이터셋을 벡터(vector) 형태로 가공하기 - 자질(feature) 추출
5. 모델 생성 및 평가하기
6. 모델 간 성능 비교하기
7. 모델 깊이 들여다보기
8. 벡터화 과정 튜닝해보기 (연습 문제)

## 실습 조교

- **장경록 (KAIST 전산학부 박사과정)**
    - 맹성현 교수님 연구실 (IR&NLP) 소속
    - Email: [kyoungrok.jang@kaist.ac.kr](mailto:kyoungrok.jang@kaist.ac.kr)
- **임도연 (KAIST 전산학부 석사과정)**
    - 맹성현 교수님 연구실 (IR&NLP) 소속
    - Email: [dylim@kaist.ac.kr](mailto:dylim@kaist.ac.kr)

## Acknowledge

본 강의는 다음 출처의 강의 노트 및 동영상을 기반으로 작성되었습니다.

[https://github.com/justmarkham/pycon-2016-tutorial](https://github.com/justmarkham/pycon-2016-tutorial)

----

In [32]:
# 파이썬 2와의 호환성을 위한 코드: print를 함수로써 사용
from __future__ import print_function

## 1단원: `scikit-learn` 맛보기
* 샘플 데이터인 iris dataset(붓꽃 데이터셋)에는 150개의 샘플이 있으며, 각 샘플은 꽃받침의 길이/너비, 꽃잎의 길이/너비 총 4가지 자질로 구성되어 있습니다.
* 이 단원에서는 주어진 자질을 이용하여 각 샘플이 3가지 붓꽃 유형 중 어떤 것에 해당되는지를 분류해 봅니다. 

In [37]:
# 예제를 위해 iris 데이터셋 로드
from sklearn.datasets import load_iris
iris = load_iris()

In [4]:
iris

{'data': array([[5.1, 3.5, 1.4, 0.2],
        [4.9, 3. , 1.4, 0.2],
        [4.7, 3.2, 1.3, 0.2],
        [4.6, 3.1, 1.5, 0.2],
        [5. , 3.6, 1.4, 0.2],
        [5.4, 3.9, 1.7, 0.4],
        [4.6, 3.4, 1.4, 0.3],
        [5. , 3.4, 1.5, 0.2],
        [4.4, 2.9, 1.4, 0.2],
        [4.9, 3.1, 1.5, 0.1],
        [5.4, 3.7, 1.5, 0.2],
        [4.8, 3.4, 1.6, 0.2],
        [4.8, 3. , 1.4, 0.1],
        [4.3, 3. , 1.1, 0.1],
        [5.8, 4. , 1.2, 0.2],
        [5.7, 4.4, 1.5, 0.4],
        [5.4, 3.9, 1.3, 0.4],
        [5.1, 3.5, 1.4, 0.3],
        [5.7, 3.8, 1.7, 0.3],
        [5.1, 3.8, 1.5, 0.3],
        [5.4, 3.4, 1.7, 0.2],
        [5.1, 3.7, 1.5, 0.4],
        [4.6, 3.6, 1. , 0.2],
        [5.1, 3.3, 1.7, 0.5],
        [4.8, 3.4, 1.9, 0.2],
        [5. , 3. , 1.6, 0.2],
        [5. , 3.4, 1.6, 0.4],
        [5.2, 3.5, 1.5, 0.2],
        [5.2, 3.4, 1.4, 0.2],
        [4.7, 3.2, 1.6, 0.2],
        [4.8, 3.1, 1.6, 0.2],
        [5.4, 3.4, 1.5, 0.4],
        [5.2, 4.1, 1.5, 0.1],
  

In [40]:
# 자질 행렬 (X)과 예측값 벡터 (y)를 변수에 저장
X = iris.data
y = iris.target

- **"자질"**은 데이터의 특성을 나타내는 정보이며, 입력값 혹은 속성이라고도 불립니다.
- **"예측값"**은 자질에 기반해 예측하고자 하는 값입니다 (예: 주어진 이메일이 스팸인지 여부)

In [13]:
# X와 y의 데이터 모양(shape) 확인
# 자질 행렬 X의 경우 '행'이 데이터 개수, '열'이 자질의 개수를 나타냅니다
print(X.shape)
print(y.shape)

(150, 4)
(150,)


데이터의 어떤 형태로 구조화되어 있는지 확인해 보겠습니다.

우선 자질 행렬인 `X`의 첫 5행을 확인합니다. 각 행은 4개의 자질로 이루어져 있습니다.

In [38]:
# 첫 5행의 샘플 확인
import pandas as pd
pd.DataFrame(X, columns=iris.feature_names).head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)


다음은 예측값 벡터인 `y`의 모습입니다. 붓꽃의 유형은 3가지이므로 각 요소는 (0, 1, 2) 중 하나의 값을 가지고 있습니다.

In [8]:
# 예측값 벡터 확인
print(y)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


`y`의 길이는 샘플의 개수인 150개와 일치합니다.

In [9]:
len(y)

150

* 입력값이 되는 자질을 숫자로 표현할 수 있어야 (분류)모델을 만드는데 활용할 수 있습니다.
* 또한 모든 샘플은 **동일한 순서의 자질**로 구성되어 있어야 합니다.

이제 위에서 준비한 자질과 예측값을 가지고 **k-최근접 이웃 모델(k-NN)**을 학습해 보겠습니다.

![k-NN](./assets/knn.png)

이미지 출처: https://www.cogneta.ai/blog/2017/11/11/choosing-the-right-machine-learning-model-part-2-k-nearest-neighbor

In [41]:
# 모델 클래스 로드
from sklearn.neighbors import KNeighborsClassifier

# 모델 초기화
knn = KNeighborsClassifier()

# 모델 학습 (`X`, `y` 활용)
knn.fit(X, y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')

학습된 모델을 가지고 새로운 입력값이 어떤 붓꽃 유형에 해당되는지 예측해 보겠습니다.

In [42]:
# 새로운 붓꽃 샘플이 어떤 유형에 속하는지 예측
knn.predict([[1, 100, 5, 2]])[0]

0

## 2단원: 텍스트를 연산 가능한 형태로 변환하기

`simple_train`은 예시로 사용할 데이터입니다. 3개의 문장으로 이루어져 있습니다.
* "call you tonight"
* "Call me a cab"
* "please call me... PLEASE!"

In [1]:
simple_train = ['call you tonight', 'Call me a cab', 'please call me... PLEASE!']

위의 데이터는 그대로 모델의 입력값으로 사용할 수 없습니다. 왜냐하면 대다수의 모델이 **숫자값으로 구성되고 크기가 일정한 벡터**가 입력되기를 기대하는데 반해, 위 문장들은 **가공되지 않은 텍스트로 구성되었고 크기가 일정하지 않기** 때문입니다.
(ref: [scikit-learn documentation](http://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction))

그렇기 때문의 scikit-learn의 [CountVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)를 활용하여 위 문장들을 **구성하는 단어의 개수로 구성된 행렬**로 변환합니다.

In [2]:
# CountVectorizer 로드
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()

In [3]:
# 예시 데이터를 어떻게 수치적으로 변환할지 학습 (어휘 사전 학습)
vect.fit(simple_train)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

**Q: 위 문장에 있던 단어 중 어떤 것들이 아래 어휘 사전에 학습되었나요? 모든 단어가 포함되었나요?**
* `['cab', 'call', 'me', 'please', 'tonight', 'you']`

In [4]:
import pandas as pd

In [5]:
# 학습된 어휘 사전 확인
vect.get_feature_names()

['cab', 'call', 'me', 'please', 'tonight', 'you']

In [6]:
# 어휘 사전에 기반하여 예제 데이터를 숫자로 구성된 행렬로 변환
simple_train_dtm = vect.transform(simple_train)
simple_train_dtm

<3x6 sparse matrix of type '<class 'numpy.int64'>'
	with 9 stored elements in Compressed Sparse Row format>

변환된 행렬을 확인해 보겠습니다.

In [7]:
# 변환된 행렬 확인
simple_train_dtm.toarray()

array([[0, 1, 0, 0, 1, 1],
       [1, 1, 1, 0, 0, 0],
       [0, 1, 1, 2, 0, 0]], dtype=int64)

In [8]:
simple_train_dtm.toarray().sum()

10

In [9]:
simple_train = ['call you tonight', 'Call me a cab', 'please call me... PLEASE!']

In [10]:
pd.DataFrame(simple_train_dtm.toarray(), columns=vect.get_feature_names())

,cab,call,me,please,tonight,you
0,0,1,0,0,1,1
1,1,1,1,0,0,0
2,0,1,1,2,0,0


[scikit-learn documentation](http://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction)에서 발췌:

> 위의 경우 자질과 샘플은 다음과 같이 정의 가능하다:

> - **자질**: 각 단어의 출현 빈도
> - **샘플**: 주어진 문장(혹은 문서)를 어휘에 있는 모든 단어의 출현 빈도(i.e. 자질)로 표현한 것

> 따라서 다수의 문서는 한 행이 고유한 문서 하나, 한 열이 고유한 단어 하나를 나타내는 행렬로 표현 가능하다

> 텍스트를 이러한 수치적 행렬로 변환하는 과정을 **벡터화(vectorization)**라고 표현하며, 위처럼 단어의 순서를 무시한 상태로 벡터화하는 방식을   
> **"Bag of Words (BoW)"**라고 부름.

In [13]:
# check the type of the document-term matrix
type(simple_train_dtm)

scipy.sparse.csr.csr_matrix

In [14]:
# examine the sparse matrix contents
print(simple_train_dtm)

  (0, 1)	1
  (0, 4)	1
  (0, 5)	1
  (1, 0)	1
  (1, 1)	1
  (1, 2)	1
  (2, 1)	1
  (2, 2)	1
  (2, 3)	2


[scikit-learn documentation](http://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction)에서 발췌:

> 대부분의 문서는 전체 어휘 중 일부만으로 구성될 것이므로, 각 문서를 나타내는 벡터 요소 중 대부분은 0의 값을 가지게 됨 (일반적으로 99% 이상).

> 가령 전체 어휘가 10만 단어라고 한다면, 각 문서를 구성하는 벡터 중 99,000개의 요소는 0의 값을, 나머지 1,000개의 요소는 1 이상의 값을 가지게 됨.

> 이렇게 저장하는 것은 저장공간/연산량 측면에서 비효율적이므로 일반적으로 **희소 행렬**(듬성듬성한 행렬) 형태로 저장하는 방식을 택함. 0이 아닌 요소만 골라 저장하는 방식. `scipy.sparse`가 그러한 구현체 중 하나.

또다른 예제를 보겠습니다.

In [15]:
# 또다른 예제
simple_test = ["please don't call me"]

In [16]:
# 예제 데이터를 행렬 형태로 변환
simple_test_dtm = vect.transform(simple_test)
simple_test_dtm.toarray()

array([[0, 1, 1, 1, 0, 0]], dtype=int64)

모든 샘플은 동일한 순서의 자질로 구성되어야 합니다.
* `['cab', 'call', 'me', 'please', 'tonight', 'you']`

In [17]:
# 어휘, 행렬 함께 보기
pd.DataFrame(simple_test_dtm.toarray(), columns=vect.get_feature_names())

,cab,call,me,please,tonight,you
0,0,1,1,1,0,0


**요약:**

- `vect.fit(train)`은 전체 데이터셋을 구성하는 **어휘 사전**을 학습합니다.
- `vect.transform(train)`은 학습된 어휘 사전에 기반하여 데이터셋을 수치적 **행렬 형태로 변환**합니다.
- `vect.transform(test)`은 새로운 데이터셋을 수치적 행렬 형태로 변환합니다 (어휘 사전에 없는 단어는 무시)

## 3단원: `pandas`를 활용하여 텍스트 데이터셋 구조화하기

In [18]:
# pandas를 이용하여 파일 로드
path = 'data/sms.tsv'
sms = pd.read_table(path, header=None, names=['label', 'message'])

In [19]:
import numpy as np

데이터 개수 & 자질 개수 확인

In [20]:
# 데이터 모양 확인
sms.shape

(5572, 2)

첫 10행 확인

In [21]:
# examine the first 10 rows
sms.head(10)

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


예측값(label)의 분포 확인
* **ham**: 스팸이 아님 
    * 4825개
* **spam**: 스팸 
    * 747개

In [22]:
# examine the class distribution
sms.label.value_counts()

ham     4825
spam     747
Name: label, dtype: int64

예측값을 수치적으로 변환 (0 혹은 1) 
* `label_num`

In [23]:
# convert label to a numerical variable
sms['label_num'] = np.where(sms['label']=='ham',0,1)

# check that the conversion worked
sms.head(10)

,label,message,label_num
0,ham,"Go until jurong point, crazy.. Available only ...",0
1,ham,Ok lar... Joking wif u oni...,0
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,ham,U dun say so early hor... U c already then say...,0
4,ham,"Nah I don't think he goes to usf, he lives aro...",0
5,spam,FreeMsg Hey there darling it's been 3 week's n...,1
6,ham,Even my brother is not like to speak with me. ...,0
7,ham,As per your request 'Melle Melle (Oru Minnamin...,0
8,spam,WINNER!! As a valued network customer you have...,1
9,spam,Had your mobile 11 months or more? U R entitle...,1


### 잠시 **붓꽃 데이터셋**의 구성과 지금의 **SMS 데이터셋**의 구성을 비교해 보겠습니다.

### 붓꽃 데이터셋
* `X`: 150개 샘플, 4개의 자질 (꽃받침 및 꽃잎의 길이/너비)
* `y`: 150개의 예측값 (붓꽃 유형)

In [59]:
# how to define X and y (from the iris data) for use with a MODEL
X = iris.data
y = iris.target
print(X.shape)
print(y.shape)

(150, 4)
(150,)


### SMS 데이터셋
* `X`: 5572개의 샘플, 1개의 자질 (문장 - 아직 행렬로 변환되지 않음)
* `y`: 5572개의 예측값 (스팸 여부)

In [24]:
# how to define X and y (from the SMS data) for use with COUNTVECTORIZER
X = sms.message
y = sms.label_num
print(X.shape)
print(y.shape)

(5572,)
(5572,)


### SMS 데이터셋을 훈련셋/테스트셋으로 분할합니다 
* 훈련셋 4179개 / 테스트셋 1393개
    * 3:1 비율
* **Q: 훈련셋? 테스트셋?**

In [25]:
# split X and y into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(4179,)
(1393,)
(4179,)
(1393,)


## 4단원: SMS 데이터셋을 벡터(vector) 형태로 가공하기

In [26]:
# CountVectorizer 초기화
vect = CountVectorizer()

In [28]:
# 훈련셋의 어휘 학습 및 벡터화 수행
vect.fit(X_train)
X_train_dtm = vect.transform(X_train)

In [30]:
pd.set_option("display.max_columns",None)

In [31]:
train_vector = pd.DataFrame(X_train_dtm.toarray(), columns=vect.get_feature_names())
train_vector.head()

00  000  008704050406  0121  01223585236  01223585334  0125698789  02  \
0   0    0             0     0            0            0           0   0   
1   0    0             0     0            0            0           0   0   
2   0    0             0     0            0            0           0   0   
3   0    0             0     0            0            1           0   0   
4   0    0             0     0            0            0           0   0   

   0207  02072069400  02073162414  02085076972  021  03  04  0430  05  050703  \
0     0            0            0            0    0   0   0     0   0       0   
1     0            0            0            0    0   0   0     0   0       0   
2     0            0            0            0    0   0   0     0   0       0   
3     0            0            0            0    0   0   0     0   0       0   
4     0            0            0            0    0   0   0     0   0       0   

   0578  06  07  07008009200  07090201529  07090298926  07123456789  \
0     0   0   0            0            0            0            0   
1     0   0   0            0            0            0            0   
2     0   0   0            0            0            0            0   
3     0   0   0            0            0            0            0   
4     0   0   0            0            0            0            0   

   07732584351  07734396839  07742676969  0776xxxxxxx  07781482378  \
0            0            0            0            0            0   
1            0            0            0            0            0   
2            0            0            0            0            0   
3            0            0            0            0            0   
4            0            0            0            0            0   

   07786200117  078  07801543489  07808  07808247860  07808726822  \
0            0    0            0      0            0            0   
1            0    0            0      0            0            0   
2            0    0            0      0            0            0   
3            0    0            0      0            0            0   
4            0    0            0      0            0            0   

   07815296484  07821230901  07880867867  0789xxxxxxx  07946746291  \
0            0            0            0            0            0   
1            0            0            0            0            0   
2            0            0            0            0            0   
3            0            0            0            0            0   
4            0            0            0            0            0   

   0796xxxxxx  07973788240  07xxxxxxxxx  08  0800  08000407165  08000776320  \
0           0            0            0   0     0            0            0   
1           0            0            0   0     0            0            0   
2           0            0            0   0     0            0            0   
3           0            0            0   0     0            0            0   
4           0            0            0   0     0            0            0   

   08000839402  08000930705  08000938767  08001950382  08002888812  \
0            0            0            1            0            0   
1            0            0            0            0            0   
2            0            0            0            0            0   
3            0            0            0            0            0   
4            0            0            0            0            0   

   08002986030  08002986906  08002988890  08006344447  0808  08081263000  \
0            0            0            0            0     0            0   
1            0            0            0            0     0            0   
2            0            0            0            0     0            0   
3            0            0            0            0     0            0   
4            0            0            0            0     0            0   

   08081560665  0825  

In [32]:
train_vector.shape

(4179, 7456)

In [33]:
# (위의 동작과 동일) `fit_transform`: 어휘 학습 및 벡터화를 동시에 수행
X_train_dtm = vect.fit_transform(X_train)

`X_train_dtm`: `CountVectorizer`로 변환된 훈련셋 행렬

In [34]:
# 행렬 생성 확인
X_train_dtm

<4179x7456 sparse matrix of type '<class 'numpy.int64'>'
	with 55209 stored elements in Compressed Sparse Row format>

In [35]:
# 테스트셋도 변환
X_test_dtm = vect.transform(X_test)
X_test_dtm

<1393x7456 sparse matrix of type '<class 'numpy.int64'>'
	with 17604 stored elements in Compressed Sparse Row format>

In [37]:
test_vector = pd.DataFrame(X_test_dtm.toarray(), columns=vect.get_feature_names())
test_vector.head()

00  000  008704050406  0121  01223585236  01223585334  0125698789  02  \
0   0    0             0     0            0            0           0   0   
1   0    0             0     0            0            0           0   0   
2   0    0             0     0            0            0           0   0   
3   0    0             0     0            0            0           0   0   
4   0    0             0     0            0            0           0   0   

   0207  02072069400  02073162414  02085076972  021  03  04  0430  05  050703  \
0     0            0            0            0    0   0   0     0   0       0   
1     0            0            0            0    0   0   0     0   0       0   
2     0            0            0            0    0   0   0     0   0       0   
3     0            0            0            0    0   0   0     0   0       0   
4     0            0            0            0    0   0   0     0   0       0   

   0578  06  07  07008009200  07090201529  07090298926  07123456789  \
0     0   0   0            0            0            0            0   
1     0   0   0            0            0            0            0   
2     0   0   0            0            0            0            0   
3     0   0   0            0            0            0            0   
4     0   0   0            0            0            0            0   

   07732584351  07734396839  07742676969  0776xxxxxxx  07781482378  \
0            0            0            0            0            0   
1            0            0            0            0            0   
2            0            0            0            0            0   
3            0            0            0            0            0   
4            0            0            0            0            0   

   07786200117  078  07801543489  07808  07808247860  07808726822  \
0            0    0            0      0            0            0   
1            0    0            0      0            0            0   
2            0    0            0      0            0            0   
3            0    0            0      0            0            0   
4            0    0            0      0            0            0   

   07815296484  07821230901  07880867867  0789xxxxxxx  07946746291  \
0            0            0            0            0            0   
1            0            0            0            0            0   
2            0            0            0            0            0   
3            0            0            0            0            0   
4            0            0            0            0            0   

   0796xxxxxx  07973788240  07xxxxxxxxx  08  0800  08000407165  08000776320  \
0           0            0            0   0     0            0            0   
1           0            0            0   0     0            0            0   
2           0            0            0   0     0            0            0   
3           0            0            0   0     0            0            0   
4           0            0            0   0     0            0            0   

   08000839402  08000930705  08000938767  08001950382  08002888812  \
0            0            0            0            0            0   
1            0            0            0            0            0   
2            0            0            0            0            0   
3            0            0            0            0            0   
4            0            0            0            0            0   

   08002986030  08002986906  08002988890  08006344447  0808  08081263000  \
0            0            0            0            0     0            0   
1            0            0            0            0     0            0   
2            0            0            0            0     0            0   
3            0            0            0            0     0            0   
4            0            0            0            0     0            0   

   08081560665  0825  

In [38]:
test_vector.shape

(1393, 7456)

## 5단원: 분류 모델 생성 및 평가하기

본 실습에서는 분류를 위해 나이브 베이즈 모델 ([Naive Bayes](http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.MultinomialNB.html))을 사용합니다. 자세한 내용은 실습 노트를 참고해주세요.

In [39]:
# 모델 초기화
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

In [40]:
# X_train_dtm을 활용하여 모델 학습 (대략 2 밀리초 소요)
nb.fit(X_train_dtm, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [41]:
# 테스트셋에 대한 예측 수행
y_pred_class = nb.predict(X_test_dtm)

In [42]:
y_pred_class

array([0, 0, 0, ..., 0, 1, 0])

In [43]:
# 예측값의 정확도 확인 (약 98.85%)
from sklearn import metrics
metrics.accuracy_score(y_test, y_pred_class)

0.9885139985642498

아래는 오차행렬(confusion matrix)의 모습입니다. 
* 분류 성공: 
    * 1202개의 ham
    * 174개의 spam
* 분류 실패: 
    * 6개의 ham -> spam으로 잘못 분류 
    * 11개의 spam -> ham으로 잘못 분류

![오차행렬](./assets/confusion-matrix.png)

이미지 출처: https://www.dataschool.io/simple-guide-to-confusion-matrix-terminology/

In [44]:
# 오차행렬(confusion matrix) 확인
metrics.confusion_matrix(y_test, y_pred_class)

array([[1203,    5],
       [  11,  174]], dtype=int64)

### 오류 확인하기
**ham이 spam으로 잘못 분류된 케이스 확인**

In [45]:
X_test[y_test < y_pred_class]

574               Waiting for your call.
3375             Also andros ice etc etc
45      No calls..messages..missed calls
3415             No pic. Please re-send.
1988    No calls..messages..missed calls
Name: message, dtype: object

**spam이 ham으로 잘못 분류된 케이스 확인**

In [46]:
X_test[y_test > y_pred_class]

3132    LookAtMe!: Thanks for your purchase of a video...
5       FreeMsg Hey there darling it's been 3 week's n...
3530    Xmas & New Years Eve tickets are now on sale f...
684     Hi I'm sue. I am 20 years old and work as a la...
1875    Would you like to see my XXX pics they are so ...
1893    CALL 09090900040 & LISTEN TO EXTREME DIRTY LIV...
4298    thesmszone.com lets you send free anonymous an...
4949    Hi this is Amy, we will be sending you a free ...
2821    INTERFLORA - It's not too late to order Inter...
2247    Hi ya babe x u 4goten bout me?' scammers getti...
4514    Money i have won wining number 946 wot do i do...
Name: message, dtype: object

In [47]:
# 잘못 분류된 케이스 하나 확인
X_test[3132]

"LookAtMe!: Thanks for your purchase of a video clip from LookAtMe!, you've been charged 35p. Think you can do better? Why not send a video in a MMSto 32323."

테스트셋의 각 샘플이 spam일 확률을 나타낸 벡터. **값의 정규화가 이루어지지 않아 값의 차이가 큰 편입니다.**

In [49]:
y_pred_prob = nb.predict_proba(X_test_dtm)[:, 1]
y_pred_prob

array([2.87744864e-03, 1.83488846e-05, 2.07301295e-03, ...,
       1.09026171e-06, 1.00000000e+00, 3.98279868e-09])

## 6단원: 모델 간 성능 비교하기

앞서 사용한 나이브 베이즈 모델과 로지스틱 회귀 모델([logistic regression](http://scikit-learn.org/stable/modules/linear_model.html#logistic-regression))의 성능을 비교해 보겠습니다.

### 나이브 베이즈 모델
![NB](./assets/naive-bayes.jpg)

이미지 출처: https://becominghuman.ai/naive-bayes-theorem-d8854a41ea08?gi=fdf168d2653e

### 로지스틱 회귀 모델
![LR](./assets/logistic-regression.png)

이미지 출처: https://www.datasciencecentral.com/profiles/blogs/why-logistic-regression-should-be-the-last-thing-you-learn-when-b

In [50]:
# 모델 초기화
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()

In [51]:
# 모델 학습
logreg.fit(X_train_dtm, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [52]:
# 테스트셋 예측
y_pred_class = logreg.predict(X_test_dtm)

테스트셋의 각 샘플이 spam일 확률을 나타낸 벡터. **값의 정규화가 잘 이루어져 있습니다.**

In [53]:
y_pred_prob = logreg.predict_proba(X_test_dtm)[:, 1]
y_pred_prob

array([0.01269556, 0.00347183, 0.00616517, ..., 0.03354907, 0.99725053,
       0.00157706])

정확도는 약 98.7%, 나이브 베이즈 모델보다 약간 낫습니다.

In [54]:
# calculate accuracy
metrics.accuracy_score(y_test, y_pred_class)

0.9877961234745154

## 7단원: 나이브 베이즈 모델 깊이 들여다보기

이 단원에서는 나이브 베이즈 모델이 어떤 단어에 기반하여 spam 여부를 판단하는지 분석해 보겠습니다.

In [55]:
# 훈련셋의 어휘 사전 크기 확인
X_train_tokens = vect.get_feature_names()
len(X_train_tokens)

7456

In [56]:
# 어휘 사전의 첫 50단어 확인
print(X_train_tokens[0:50])

['00', '000', '008704050406', '0121', '01223585236', '01223585334', '0125698789', '02', '0207', '02072069400', '02073162414', '02085076972', '021', '03', '04', '0430', '05', '050703', '0578', '06', '07', '07008009200', '07090201529', '07090298926', '07123456789', '07732584351', '07734396839', '07742676969', '0776xxxxxxx', '07781482378', '07786200117', '078', '07801543489', '07808', '07808247860', '07808726822', '07815296484', '07821230901', '07880867867', '0789xxxxxxx', '07946746291', '0796xxxxxx', '07973788240', '07xxxxxxxxx', '08', '0800', '08000407165', '08000776320', '08000839402', '08000930705']


In [57]:
# 어휘 사전의 마지막 50단어 확인
print(X_train_tokens[-50:])

['yer', 'yes', 'yest', 'yesterday', 'yet', 'yetunde', 'yijue', 'ym', 'ymca', 'yo', 'yoga', 'yogasana', 'yor', 'yorge', 'you', 'youdoing', 'youi', 'youphone', 'your', 'youre', 'yourjob', 'yours', 'yourself', 'youwanna', 'yowifes', 'yoyyooo', 'yr', 'yrs', 'ything', 'yummmm', 'yummy', 'yun', 'yunny', 'yuo', 'yuou', 'yup', 'zac', 'zaher', 'zealand', 'zebra', 'zed', 'zeros', 'zhong', 'zindgi', 'zoe', 'zoom', 'zouk', 'zyada', 'èn', '〨ud']


나이브 베이즈 모델은 각 예측값(spam/ham) 별로 각 단어가 몇번 출현했는지를 저장합니다

In [60]:
nb.feature_count_

array([[ 0.,  0.,  0., ...,  1.,  1.,  1.],
       [ 5., 23.,  2., ...,  0.,  0.,  0.]])

In [61]:
# 행은 예측값, 열은 단어를 나타냅니다
nb.feature_count_.shape

(2, 7456)

In [62]:
# HAM 메시지에서 각 단어가 몇번 출현했는지 확인
ham_token_count = nb.feature_count_[0, :]
ham_token_count

array([0., 0., 0., ..., 1., 1., 1.])

In [63]:
# SPAM 메시지에서 각 단어가 몇번 출현했는지 확인
spam_token_count = nb.feature_count_[1, :]
spam_token_count

array([ 5., 23.,  2., ...,  0.,  0.,  0.])

위 정보를 `pandas.DataFrame` 클래스를 활용하여 파악하기 쉽게 표현해 보겠습니다.

In [64]:
# create a DataFrame of tokens with their separate ham and spam counts
tokens = pd.DataFrame({'token':X_train_tokens, 'ham':ham_token_count, 'spam':spam_token_count}).set_index('token')
tokens.head()

,ham,spam
token,,
00,0.0,5.0
000,0.0,23.0
008704050406,0.0,2.0
0121,0.0,1.0
01223585236,0.0,1.0


In [65]:
# examine 5 random DataFrame rows
tokens.sample(5, random_state=6)

,ham,spam
token,,
very,64.0,2.0
nasty,1.0,1.0
villa,0.0,1.0
beloved,1.0,0.0
textoperator,0.0,2.0


나이브 베이즈 모델은 또 각 예측값이 몇번 관측되었는지도 저장합니다.

In [66]:
nb.class_count_

array([3617.,  562.])

각 단어의 "스팸을 나타내는 정도"를 계산하기 이전에, 0으로 나누는 문제를 방지하기 위한 조치를 취합니다 (모든 행에 1씩 더하기)

In [67]:
# 0으로 나누는 걸 방지하기 위해 모든 행에 1씩 더함 (smoothing)
tokens['ham'] = tokens.ham + 1
tokens['spam'] = tokens.spam + 1
tokens.sample(5, random_state=6)

,ham,spam
token,,
very,65.0,3.0
nasty,2.0,2.0
villa,1.0,2.0
beloved,2.0,1.0
textoperator,1.0,3.0


각 단어가 전체 ham/spam에서 등장한 비율을 계산합니다.

In [68]:
# convert the ham and spam counts into frequencies
tokens['ham'] = tokens.ham / nb.class_count_[0]
tokens['spam'] = tokens.spam / nb.class_count_[1]
tokens.sample(5, random_state=6)

,ham,spam
token,,
very,0.017971,0.005338
nasty,0.000553,0.003559
villa,0.000276,0.003559
beloved,0.000553,0.001779
textoperator,0.000276,0.005338


마지막으로, 각 단어의 ham : spam 비율을 계산합니다 (ham에서 등장 비율 / spam에서 등장 비율)
* `spam_ratio`

In [69]:
# calculate the ratio of spam-to-ham for each token
tokens['spam_ratio'] = tokens.spam / tokens.ham
tokens.sample(5, random_state=6)

,ham,spam,spam_ratio
token,,,
very,0.017971,0.005338,0.297044
nasty,0.000553,0.003559,6.435943
villa,0.000276,0.003559,12.871886
beloved,0.000553,0.001779,3.217972
textoperator,0.000276,0.005338,19.307829


`spam_ratio`가 높은 순으로 단어를 정렬
* 아래 코드가 작동 안할 시, `sort_values()` -> `sort()`로 변경하여 다시 실행.

In [70]:
tokens.sort_values('spam_ratio', ascending=False).head()

,ham,spam,spam_ratio
token,,,
claim,0.000276,0.158363,572.798932
prize,0.000276,0.135231,489.131673
150p,0.000276,0.087189,315.361210
tone,0.000276,0.085409,308.925267
guaranteed,0.000276,0.076512,276.745552


주어진 단어 (예: 'dating')의 `spam_ratio` 확인

In [71]:
# look up the spam_ratio for a given token
tokens.loc['dating', 'spam_ratio']

83.66725978647686

---

## 8단원: 벡터화 설정값 튜닝해보기 (연습문제 & 보고서)

* 지금까지는 [CountVectorizer](http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)의 **기본 설정값**을 사용하여 실험을 수행하였습니다. 이 설정값을 적절히 조정하여 성능 향상을 이끌어내는 것이 이번 단원의 목표입니다.

In [2]:
# CountVectorizer의 기본 설정값 확인
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer()
vect

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

벡터화를 포함하는 자질 추출 과정은 기계학습 모델의 성능을 크게 좌우합니다. 그렇기에 과업에 따른 최적의 설정값을 찾는게 매우 중요합니다.

다음은 수정할 수 있는 설정값들의 목록입니다.

- **stop_words:** {'english'}, list, 혹은 None (기본값)
    - 'english'일 경우, 내장된 영어의 stop words가 사용됨
      - stop words: 전치사 등 텍스트의 의미에 크게 영향을 주지 못하는 단어 목록. 보통 노이즈(noise)의 일종으로 간주.
    - `list`일 경우, 포함된 단어를 stop words로 간주하고 벡터화에서 제외.
    - `None`일 경우 stop words 사용 안 함.

In [3]:
# 영어 stop words 제거
vect = CountVectorizer(stop_words=['a', 'b'])

In [6]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\YOUNG\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [7]:
from nltk.corpus import stopwords # Import the stop word list 
words = [w for w in stopwords.words("english")][:20]
print(words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his']


- **ngram_range:** tuple (최대값, 최소값) 형태로 입력. 기본값=(1, 1)
  - 예: 2-gram -> 연속된 두 단어를 하나의 자질로 활용 ('waiting', 'for' -> 'wating for')

In [8]:
# 1-grams과 2-grams 사용
vect = CountVectorizer(ngram_range=(2, 3))

- **max_df:** 0.0~1.0 사이의 값. 기본값=1.0
    - df: document frequency의 약자로, 어떤 단어가 전체 문서 중 몇개의 문서에서 한번 이상 등장했는지를 나타내는 수치.
    - 설정된 `max_df` 값 이상의 df를 가지는 단어를 제거
      - 예: `max_df=0.5`일 경우, 전체 문서 중 50% 이상에서 나타나는 단어를 제거

In [9]:
vect = CountVectorizer(max_df=0.5)

- **min_df:** 0.0~1.0 사이의 값 혹은 정수. 기본값=1 (한 건 이상의 문서에서 출현)
    - `max_df`의 반대 작용을 함 (예: 0.2일 경우, 최소 20%의 문서에서 등장한 단어만을 벡터화에 사용)
    - 정수로 주어질 경우, 문서 출현 절대값으로 작용 (예: 2 -> 두 건 이상의 문서에서 출현)

In [102]:
# 2건 이상의 문서에서 출현한 경우만 남겨둠
vect = CountVectorizer(min_df=2)

In [103]:
vect = CountVectorizer(max_df=0.5, min_df=2)

----

### 보고서 제출 가이드라인

- 설정값들의 의미, 데이터의 특성을 함께 고려하여 최적의 설정값을 찾아주세요.
  - 가장 좋은 설정값 = 모델의 성능이 가장 좋을 때의 값
  - 4단원 첫 줄 (`vect = CountVectorizer()`)에 설정값을 반영 후, 끝까지 다시 실행시켜 주세요.
  - 여러 설정값을 동시에 입력해야 함께 적용됩니다.
    - 예: `vect = CountVectorizer(max_df=0.5, min_df=2)`
- **다음 정보를 조교 이메일(dylim@kaist.ac.kr)로 발송해주세요.**
  - 성함
  - 사용한 설정값 (예: `max_df=0.5`, `ngram_range=(2, 3)`)
  - 나이브 베이즈 모델의 정확도
  - 설정값이 반영된 `tutorial_{성함}.ipynb`

---